In [57]:
import pyomo.environ as pyo

In [58]:
path_solver = "../solvers/glpk-4.65/w64/glpsol.exe"

In [104]:
capacidades = [1000, 1500, 1200]
demandas = [2300, 1400]
#capacidades = [1, 1, 1]
#demandas = [1, 1]

sum(capacidades) == sum(demandas)
c_1j = [152, 247]
c_2j = [117, 193]
c_3j = [131, 185]
c_ij = [c_1j, c_2j, c_3j]
#sum(c_1j),sum(c_2j),sum(c_3j), sum(c_1j)+sum(c_2j)+sum(c_3j)
indx = len(capacidades), len(demandas)

In [105]:
indx

(3, 2)

In [106]:
modelo = pyo.ConcreteModel()

modelo.I = pyo.RangeSet(indx[0])
modelo.J = pyo.RangeSet(indx[1])

modelo.X = pyo.Var(modelo.I, modelo.J, within=pyo.NonNegativeIntegers)
modelo.C = pyo.Param(modelo.I, modelo.J, initialize=lambda model, i, j: c_ij[i-1][j-1])
modelo.Cap =pyo.Param(modelo.I, initialize=lambda model, i: capacidades[i-1])
modelo.dem =pyo.Param(modelo.J, initialize=lambda model, j: demandas[j-1])

In [107]:
def objt(model):
    #return sum(model.C[i,j]*model.X[i,j] for i in model.I for j in model.J)
    #return sum(model.C[i,j]*model.X[i,j] for i, j in zip(model.I, model.J))
    return pyo.summation(model.C, model.X)


modelo.Obj = pyo.Objective(rule=objt, sense=pyo.minimize)


In [108]:
def rest_1(model,i):
    return sum(model.X[i,j] for j in model.J) <= model.Cap[i]

def rest_2(model,j):
    return sum(model.X[i,j] for i in model.I) >= model.dem[j]

modelo.R1 = pyo.Constraint(modelo.I, rule=rest_1)
modelo.R2 = pyo.Constraint(modelo.J, rule=rest_2)

In [109]:
solver = pyo.SolverFactory('glpk', executable=path_solver)
solver.solve(modelo, tee=False)

{'Problem': [{'Name': 'unknown', 'Lower bound': 564700.0, 'Upper bound': 564700.0, 'Number of objectives': 1, 'Number of constraints': 5, 'Number of variables': 6, 'Number of nonzeros': 12, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.0337374210357666}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [110]:
modelo.Obj()

564700.0

In [111]:
for i in modelo.I:
    for j in modelo.J:
#        print(i,j,"-----",modelo.X[i,j](), end='')
        print(modelo.X[i,j](), end=' ')
    print("")

1000.0 0.0 
1300.0 200.0 
0.0 1200.0 


In [112]:
lista = list(modelo.X.keys())
for i in lista:
    if modelo.X[i]()!=0:
        print(i,'---',modelo.X[i]())

(1, 1) --- 1000.0
(2, 1) --- 1300.0
(2, 2) --- 200.0
(3, 2) --- 1200.0


## Problema de allocação de tarefas

In [83]:
import pyomo.environ as pyoEn 

In [84]:
#path_solver = "../solvers/glpk-4.65/w64/glpsol.exe"


In [154]:
#cust_1j = [152, 247]
#cust_2j = [117, 193]
#cust_3j = [131, 185]
cust_1j = [230, 400, 159]
cust_2j = [123, 21, 321]
cust_3j = [333, 214, 252]
c = [cust_1j,cust_2j,cust_3j]
n, m = len(c[0]), len(c)


In [155]:
mod = pyoEn.ConcreteModel()

mod.I = pyoEn.RangeSet(m)
mod.J = pyoEn.RangeSet(n)

mod.X = pyoEn.Var(mod.I, mod.J, within=pyoEn.Binary)

def init_cust(model, i, j):
    return c[i-1][j-1]

mod.C = pyoEn.Param(mod.I, mod.J, initialize=init_cust)


In [156]:
def objct(model):
    #return sum(model.X[i,j]*model.C[i,j] for i in model.I for j in model.J)
    return pyoEn.summation(model.C, model.X)

mod.Obj = pyoEn.Objective(rule=objct, sense=pyoEn.minimize)

In [157]:
def r1(model, i):
    return sum(model.X[i,j] for j in mod.J) == 1

def r2(model, j):
    return sum(model.X[i,j] for i in mod.I) == 1

mod.R1 = pyoEn.Constraint(mod.I, rule=r1)
mod.R2 = pyoEn.Constraint(mod.J, rule=r2)

In [158]:
solver = pyoEn.SolverFactory('glpk', executable=path_solver)
solution = solver.solve(mod, tee=False)

In [159]:
for i in mod.I:
    for j in mod.J:
#        print(i,j,"-----",modelo.X[i,j](), end='')
        print(mod.X[i,j](), end=' ')
    print("")

0.0 0.0 1.0 
1.0 0.0 0.0 
0.0 1.0 0.0 


In [160]:
#mod.pprint()
mod.Obj()


496.0

In [161]:
#for i in model.I:
#    print(f'{i}---',mod.X[i])
#
lista = list(mod.X.keys())
for i in lista:
    if mod.X[i]()!=0:
        print(i,'---',mod.X[i]())

(1, 3) --- 1.0
(2, 1) --- 1.0
(3, 2) --- 1.0


In [162]:
mod.C.pprint()

C : Size=9, Index=C_index, Domain=Any, Default=None, Mutable=False
    Key    : Value
    (1, 1) :   230
    (1, 2) :   400
    (1, 3) :   159
    (2, 1) :   123
    (2, 2) :    21
    (2, 3) :   321
    (3, 1) :   333
    (3, 2) :   214
    (3, 3) :   252
